In [1]:
import cv2
import numpy as np
import face_recognition
import os

In [2]:
path = r'C:\Users\JOKER\Desktop\face recognition'  # here collected dataset that we used for recognition
images = []                                              # Only for images
classNames = []  # only for images names  note: image withrespective name of images
myList = os.listdir(path)  #taking the path access
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')     #accessing the all images 
    images.append(curImg)                     # storing the all images
    classNames.append(os.path.splitext(cl)[0])    # storing the names of the images 

#encodeListKnown=np.load('encodeface.npy')    # loading the encoding file of the collected images it will store in array formate

In [3]:
def findEncodings(images):       #this block is used for the training the images in the digital format 
    encodeList = []              # empty list for storing the encoded images for recognition
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)            # converting images into original format 
        encode = face_recognition.face_encodings(img)[0]        # storing the encoded images in encode
        encodeList.append(encode)                               # storing the all encoded images
    return encodeList
encodeListKnown = findEncodings(images)                      # return the encoded data 
#np.save('encodeface.npy',encodeListKnown)   # here encoded data takes time so for reduce the time we save the encode data

In [17]:
cap=cv2.VideoCapture(0)    # web cam or camara accessing command
while True:
    ret,frame= cap.read()                            # here frame is the photo for every secound
    imgS = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    facesCurFrame = face_recognition.face_locations(imgS)        # loacting the face in the frame
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)  # encoded the faces in the frame
    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)   # compare the encoded face from cam and encoded faces from colleceted data
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)  # find the which position of the faces is matched such if face is matched at six face then return the 6 values in array format
            matchIndex = np.argmin(faceDis)  # convert the array to normal number such as integer format
    
            if matches[matchIndex]:     
                name = classNames[matchIndex].upper()   # finding the name via using the matchIndex number
                x,y,w,h = faceLoc        # loacation of the faces which are matched in x/4 ratio format
                cv2.rectangle(frame, (y,x),(h,w), (0, 255, 0), 2)    # creating the rectangle on matched faces
                cv2.rectangle(frame, (y,w-35),(h,w), (0, 255, 0), cv2.FILLED)  # this is used for putting the recognized name
                # box for face and box for text
                cv2.putText(frame, name, (h + 6, w - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)  # name text

    cv2.imshow('testing faces',frame)   # return the frame after recognition
    if cv2.waitKey(1) & 0xff==ord('q'):   # used for closing the loop 
        break
cap.release()        # deacitvating the cam which we used now
cv2.destroyAllWindows()   # closing the all related windows which are opened

In [ ]:
face_recognition.